In [4]:
import pandas as pd
pd.set_option('max_columns', None)
import numpy as np
import json
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import glob

In [2]:
one_race_cols = {"Driver": "Driver", "Car": "Car #", "AvgStart": "Start", "AvgMidRace": "Mid Race", "AvgFinish": "Finish",
                 "Avg Pos": "Average Position", "PassDiff": "Pass Differential", "GreenPass": "Green Passes",
                 "GreenPassed": "Green Passed", "QualityPasses": "Quality Passes",
                 "PercentQualityPasses": "% Quality Passes", "NumFastestLaps": "# Fastest Laps",
                 "LapsinTop15": "Laps in Top 15", "PercentLapsTop15": "% Laps in Top 15", "LapsLed": "Laps Led",
                 "PercentLapsLed": "% Laps Led", "TotalLaps": "Total Laps", "DriverRating": "Driver Rating",
                 "Pts": "Points"}

one_race_cols_2 = {"Driver": "Driver", "Car": "Car #", "AvgPos": "Average Position", "PassDiff": "Pass Differential",
                   "GreenPass": "Green Passes", "GreenPassed": "Green Passed", "QualityPasses": "Quality Passes",
                   "PercentQualityPasses": "% Quality Passes", "NumFastestLaps": "# Fastest Laps",
                   "LapsTop15": "Laps in Top 15", "PercentLapsTop15": "% Laps in Top 15", "LapsLed": "Laps Led",
                   "PercentLapsLed": "% Laps Led", "TotalLaps": "Total Laps", "DriverRating": "Driver Rating",
                   "Pts": "Points", "StartPos": "Start", "MidRace": "Mid Race", "FinishPos": "Finish"}

car_nums = {"AJ Allmendinger": 16, "Aric Almirola": 10, "Christopher Bell": 20, "Greg Biffle": 44,
             "Josh Bilicki": 77, "Ryan Blaney": 12, "Alex Bowman": 48, "Chase Briscoe": 14,
             "Chris Buescher": 17, "Harrison Burton": 21, "Kurt Busch": 45, "Kyle Busch": 18,
             "William Byron": 24, "Landon Cassill": 77, "Ross Chastain": 1,
             "Austin Cindric": 2, "Cole Custer": 41, "Austin Dillon": 3, "Ty Dillon": 42,
             "Chase Elliott": 9, "Todd Gilliland": 38, "Noah Gragson": 16, "Kaz Grala": 50,
             "Denny Hamlin": 11, "Joey Hand": 15, "Kevin Harvick": 4, "Justin Haley": 31,
             "Daniel Hemric": 16, "Loris Hezemans": 27, "Erik Jones": 20, "Brad Keselowski": 6,
             "Corey LaJoie": 7, "Kyle Larson": 5, "Joey Logano": 22, "BJ McLeod": 78,
             "Michael McDowell": 34, "Ryan Preece": 15, "David Ragan": 15, "Tyler Reddick": 8,
             "Boris Said": 66, "Garrett Smithley": 15, "Ricky Stenhouse Jr.": 47, "Daniel Suárez": 99,
             "Martin Truex Jr.": 19, "Jacques Villeneuve": 27, "Bubba Wallace": 23, "Cody Ware": 51}

teams = {"AJ Allmendinger": "Kaulig", "Aric Almirola": "SHR", "Christopher Bell": "JGR", "Greg Biffle": "NYR",
        "Josh Bilicki": "Spire", "Ryan Blaney": "Penske", "Alex Bowman": "Hendrick", "Chase Briscoe": "SHR",
        "Chris Buescher": "RFK", "Harrison Burton": "WBR", "Kurt Busch": "23X1", "Kyle Busch": "JGR",
        "William Byron": "Hendrick", "Landon Cassill": "Spire", "Ross Chastain": "Trackhouse", "Austin Cindric": "Penske",
        "Cole Custer": "SHR", "Austin Dillon": "RCR", "Ty Dillon": "PGMS", "Chase Elliott": "Hendrick",
        "Todd Gilliland": "FRM", "Noah Gragson": "Kaulig", "Kaz Grala": "TMT", "Denny Hamlin": "JGR",
        "Joey Hand": "RWR", "Kevin Harvick": "SHR", "Justin Haley": "Kaulig", "Daniel Hemric": "Kaulig",
        "Loris Hezemans": "Hezeberg", "Erik Jones": "PGMS", "Brad Keselowski": "RFK", "Corey LaJoie": "Spire",
        "Kyle Larson": "Hendrick", "Joey Logano": "Penske", "BJ McLeod": "LFM", "Michael McDowell": "FRM",
        "Ryan Preece": "RWR", "David Ragan": "RWR", "Tyler Reddick": "RCR", "Boris Said": "MBM",
        "Garrett Smithley": "RWR", "Ricky Stenhouse Jr.": "JTGD", "Daniel Suárez": "Trackhouse", "Martin Truex Jr.": "JGR",
        "Jacques Villeneuve": "Hezeberg", "Bubba Wallace": "23X1", "Cody Ware": "RWR"}

makes = {"23X1": "Toyota", "Beard": "Chevy", "FRM": "Ford", "Hendrick": "Chevy",
        "Hezeberg": "Ford", "JGR": "Toyota", "JTGD": "Chevy", "Kaulig": "Chevy",
        "LFM": "Ford", "MBM": "Ford", "NYR": "Chevy", "Penske": "Ford",
        "PGMS": "Chevy", "RCR": "Chevy", "RFK": "Ford", "RWR": "Ford",
        "SHR": "Ford", "Spire": "Chevy", "TMT": "Chevy", "Trackhouse": "Chevy",
        "WBR": "Ford"}

track_types = {"Auto Club": "Speedway", "Atlanta": "Superpeedway", "Bristol": "Short Track", "COTA": "Road Course",
               "Daytona": "Superspeedway", "Las Vegas": "Speedway", "Martinsville": "Short Track", "Phoenix": "Speedway",
               "Richmond": "Short Track", "Talladega": "Superspeedway"}

cup_laps = {1: 201}

In [58]:
import re
data_path = r'C:\Users\kchen\OneDrive\MADS\SIADS591_592_Milestone1\Milestone_I\Milestone_I\data\loop_TobyChristie\csv\2022'

filenames = glob.glob(data_path + "/*.csv")
loop_data=[]
for file in filenames:
    df=pd.read_csv(file)
    race=re.search(r'(\d{8})_(\w*)',file)
    df['Date']=race.group(1)
    df['Race']=race.group(2)
    loop_data.append(df)
df=pd.concat(loop_data,ignore_index=True)

In [62]:
df

,Car,Driver,AvgStart,AvgMidRace,AvgFinish,Avg Pos,PassDiff,GreenPass,GreenPassed,QualityPasses,PercentQualityPasses,NumFastestLaps,LapsinTop15,PercentLapsTop15,LapsLed,PercentLapsLed,TotalLaps,DriverRating,Pts,Date,Race,StartPos,MidRace,CloserPos,FinishPos,HighPos,LowPos,AvgPos,LapsTop15
0,2,Austin Cindric,5.0,2.0,1.0,6.2,-13,168,181,125,74.40,0,176.0,87.6,21,10.4,201,122.4,54,20220220,Daytona500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,6,Brad Keselowski,3.0,5.0,9.0,4.8,1,171,170,148,86.50,4,198.0,98.5,67,33.3,201,121.3,54,20220220,Daytona500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,19,Martin Truex Jr,14.0,18.0,13.0,11.9,25,258,233,192,74.40,4,143.0,71.1,11,5.5,201,94.0,49,20220220,Daytona500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,23,Bubba Wallace,16.0,19.0,2.0,10.2,16,253,237,191,75.50,7,145.0,72.1,12,6.0,201,104.7,47,20220220,Daytona500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,14,Chase Briscoe,9.0,10.0,3.0,18.3,17,209,192,70,33.50,6,88.0,43.8,0,0.0,201,75.0,41,20220220,Daytona500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
368,44,Greg Biffle,NaN,NaN,NaN,NaN,1,137,136,0,0.00,2,NaN,0.0,0,0.0,79,29.1,2,20220424,TalladegaSuperspeedway,35.0,35.0,35.0,35.0,17.0,37.0,31.6,0.0
369,16,Daniel Hemric,NaN,NaN,NaN,NaN,-13,71,84,68,95.77,0,NaN,29.8,0,0.0,56,74.3,0,20220424,TalladegaSuperspeedway,3.0,36.0,36.0,36.0,3.0,17.0,27.0,56.0
370,14,Chase Briscoe,NaN,NaN,NaN,NaN,0,137,137,11,8.03,6,NaN,1.6,0,0.0,56,35.2,1,20220424,TalladegaSuperspeedway,17.0,37.0,37.0,37.0,14.0,38.0,33.7,3.0
371,17,Chris Buescher,NaN,NaN,NaN,NaN,12,134,122,16,11.94,3,NaN,2.1,0,0.0,56,35.9,1,20220424,TalladegaSuperspeedway,30.0,38.0,38.0,38.0,12.0,35.0,33.7,4.0
